## PDFから指定されたセクションを抽出する

In [209]:
import fitz  # PyMuPDF
import re
from pathlib import Path
import pandas as pd
import requests
from io import BytesIO



In [210]:

# 1. PDF -> テキスト結合
def extract_text_all(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    texts = []
    for page in doc:
        texts.append(page.get_text("text"))
    return "\n".join(texts)


In [211]:
HEADING_PATTERNS = [
    r'^\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\.\-．、.][0-9０-９一二三四五六七八九十]+)*[\s　]*(教訓|提言)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]*[\s　]*(教訓|提言)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\.\-．、.][0-9０-９一二三四五六七八九十]+)*[\s　]*(教訓・提言)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]*[\s　]*(教訓・提言)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\.\-．、.][0-9０-９一二三四五六七八九十]+)*[\s　]*(教訓•提言)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]*[\s　]*(教訓•提言)',
    r'(教訓・提言)',
]

In [212]:
import re

def is_noise_paragraph(text: str) -> bool:
    s = text.strip()

    # 1. まず改行を消した短さチェック用文字列
    s_compact = s.replace("\n", "").strip()

    # 1. 極端に短い
    if len(s_compact) < 8:
        return True

    # 2. 各行が「提言」「教訓」「結論」などだけの行ばかり → ノイズ
    lines = [l.strip() for l in s.splitlines() if l.strip()]
    if lines:
        if all(re.fullmatch(r'[・･\-\.\s　]*(提言|教訓|結論)[・･\-\.\s　]*', l) for l in lines):
            return True

    # 3. 全体で「提言」「教訓」「結論」の繰り返しだけ
    compressed = re.sub(r'[・･\-\.\s　\n]+', '', s)
    if re.fullmatch(r'(提言|教訓|結論)+', compressed):
        return True

    # 4. 脚注っぽい（数字+短いテキスト）
    if re.match(r'^[0-9０-９]+\s', s_compact) and len(s_compact) < 25:
        return True

    # 5. 「主要計画／実績比較」など表見出しブロック
    #   - 主要計画 を含み、
    #   - 「実績比較」か「項  目」「計  画」「実  績」など典型的なカラム名が並んでいる
    if "主要計画" in s:
        if ("実績比較" in s
            or "項  目" in s
            or "計  画" in s
            or "実  績" in s):
            return True

    # 6. 明らかに表形式ぽい：句点がなく、改行だらけ、名詞だけ
    if "。" not in s and len(lines) >= 3:
        # かなり強めに表・見出し臭いものは弾く
        header_keywords = ["項  目", "計  画", "実  績", "指標", "区分"]
        if any(k in s for k in header_keywords):
            return True

    return False

In [213]:
import re

def is_heading_line(s: str) -> bool:
    """数字やローマ数字＋区切り記号＋短いテキスト（句点なし）を見出し候補とみなす。"""
    s = s.strip()
    if "。" in s:
        return False
    return bool(re.match(
        r'^[\s　]*[0-9０-９IVX一二三四五六七八九十]+(?:[\.．\-、][0-9０-９一二三四五六七八九十]+)*[\s　]+.{1,40}$',
        s
    ))


def find_kadai_section(full_text: str) -> str | None:
    lines = full_text.splitlines()

    # 教訓・提言セクション開始を探す
    start_idx = None
    for i, line in enumerate(lines):
        normalized = line.strip()
        for pat in HEADING_PATTERNS:
            if re.search(pat, normalized):
                start_idx = i
                break
        if start_idx is not None:
            break
    if start_idx is None:
        return None

    # 終了条件：
    #  (1) 「以上」「以 上」「以　上」「(以上)」「（以 上）」など
    #  (2) 「コラム」や「主要計画」を含む行
    #  (3) 別の見出し行
    end_idx = len(lines)
    END_LINE_PAT = r'^[\s　]*[（(]?\s*以[\s　]*上\s*[)）]?[\s　]*$'

    for j in range(start_idx + 1, len(lines)):
        line = lines[j].strip()

        # (1) 以上系
        if re.search(END_LINE_PAT, line):
            end_idx = j
            break

        # --- 修正版: 主要計画（全角・空白・スラッシュなど含む） ---
        if re.search(r'主[\s　]*要[\s　]*計[\s　]*画', line):
            end_idx = j
            break

        # --- 修正版: 主要計画（全角・空白・スラッシュなど含む） ---
        if re.search(r'コ[\s　]*ラ[\s　]*ム', line):
            end_idx = j
            break

    section_lines = lines[start_idx:end_idx]
    return "\n".join(section_lines)

In [214]:
import re

def is_page_number_line(s: str) -> bool:
    s = s.strip()
    # 完全に数字だけ / 前後に-や()だけの数字 → ページ番号とみなす
    if re.fullmatch(r'[0-9０-９]+', s):
        return True
    if re.fullmatch(r'[-–－\-（(]?\s*[0-9０-９]+\s*[-–－\-）)]?', s):
        return True
    return False


def split_into_paragraphs(section_text: str) -> list[str]:
    """
    セクションテキストを段落ごとに分割する。
    - ページ番号行を事前に削除
    - その後、ページまたぎで切れた短い行を前の段落にマージ
    """

    # 1) ページ番号行を削除
    cleaned_lines = []
    for line in section_text.splitlines():
        if is_page_number_line(line):
            continue
        cleaned_lines.append(line)

    cleaned_text = "\n".join(cleaned_lines)

    # 2) 空行ベースでざっくり段落分割
    raw_blocks = re.split(r'\n\s*\n', cleaned_text)
    paras = []
    for blk in raw_blocks:
        p = blk.strip()
        if not p:
            continue
        # 見出しカスみたいな極端に短いものは除外（必要なら調整）
        if len(p) < 10:
            continue
        paras.append(p)

    # 3) 「ページまたぎで分裂した短い行」を前の段落にマージ
    merged: list[str] = []
    for p in paras:
        if not merged:
            merged.append(p)
            continue

        prev = merged[-1]

        # マージ条件：
        # - 前の段落が「。」「！」「？」で終わっていない
        # - 今の段落が比較的短い（例: 40文字以下）
        # - 今の段落が見出しっぽくない（行頭が数字や「第」「●」などで始まらない）
        if (not re.search(r'[。！？]$', prev)
                and len(p) <= 40
                and not re.match(r'^[0-9０-９第・●･\-\(（]', p.strip())):
            # 前の段落にくっつける（改行を入れるかどうかは好み）
            merged[-1] = prev.rstrip() + p.lstrip()
        else:
            merged.append(p)

    return merged

In [215]:
def load_pdf_text(source: str) -> str:
    """
    source が
      - http(s) で始まる場合: Web上のPDFとして取得
      - それ以外: ローカルパスとして扱う
    全ページのテキストを結合して返す。
    """
    if source.startswith("http://") or source.startswith("https://"):
        # URLからPDFを取得
        resp = requests.get(source, timeout=30)
        resp.raise_for_status()
        pdf_bytes = resp.content
        doc = fitz.open(stream=BytesIO(pdf_bytes), filetype="pdf")
    else:
        # ローカルファイルとして開く
        doc = fitz.open(source)

    texts = []
    for page in doc:
        texts.append(page.get_text("text"))
    doc.close()
    return "\n".join(texts)

In [216]:
def extract_kadai_paragraphs(source: str,
                             project_id: str | None = None) -> pd.DataFrame:
    """
    1つの評価報告書（URL or ローカルパス）から、
    「教訓・提言」系セクションの段落を抜き出して DataFrame を返す。

    columns:
        project_id, para_id, text, source
    """
    full_text = load_pdf_text(source)
    kadai_section = find_kadai_section(full_text)

    if kadai_section is None:
        print(f"[WARN] 教訓・提言セクションが見つからない: {source}")
        return pd.DataFrame(columns=["project_id", "para_id", "text", "source"])

    paras = split_into_paragraphs(kadai_section)

    # 一旦 DataFrame に
    if project_id is None:
        if source.startswith("http"):
            project_id = Path(source).name  # ファイル名部分だけ
        else:
            project_id = Path(source).name

    df = pd.DataFrame(
        {
            "project_id": project_id,
            "para_id": list(range(len(paras))),
            "text": paras,
            "source": source,
        }
    )

    # ノイズ段落削除
    df = df[~df["text"].apply(is_noise_paragraph)].reset_index(drop=True)

    # para_id を振り直し
    df["para_id"] = range(len(df))

    return df


In [217]:
except_file_list = [
    # フォーマットが特殊
    'https://www2.jica.go.jp/ja/evaluation/pdf/2010_VNXI-3_4_f.pdf'
]

In [ ]:
df = pd.read_csv("../df_check_99.csv")
target_urls = df["file"].dropna().unique().tolist()
#target_urls = target_urls[0:200]  # テスト用

#target_urls = ["https://www2.jica.go.jp/ja/evaluation/pdf/2010_C01-P160_4_f.pdf"]
df_out = pd.DataFrame()
for i,url in enumerate(target_urls):
    if url in except_file_list:
        print(f"=== {i} {url} SKIPPED ===")
        continue
    df_kadai = extract_kadai_paragraphs(url)
    print(f"=== {i} {url} ===")
    df_out = pd.concat([df_out, df_kadai], ignore_index=True)


=== 0 https://www2.jica.go.jp/ja/evaluation/pdf/2010_0200600_4_f.pdf ===
=== 1 https://www2.jica.go.jp/ja/evaluation/pdf/2010_0202100_4_f.pdf ===


In [ ]:
# クレンジング
import re

def is_noise_paragraph(text: str) -> bool:
    s = str(text).strip()
    s_compact = s.replace("\n", "").strip()

    # 1. 極端に短い
    if len(s_compact) < 8:
        return True

    # 2. 教訓・提言・結論だけが並んでる系
    lines = [l.strip() for l in s.splitlines() if l.strip()]
    if lines:
        if all(re.fullmatch(r'[・･\-\.\s　]*(提言|教訓|結論)[・･\-\.\s　]*', l) for l in lines):
            return True

    # 3. 全体でも「提言/教訓/結論」だけ
    compressed = re.sub(r'[・･\-\.\s　\n]+', '', s)
    if re.fullmatch(r'(提言|教訓|結論)+', compressed):
        return True

    # 4. 脚注系（数字＋短文）
    if re.match(r'^[0-9０-９]+\s', s_compact) and len(s_compact) < 25:
        return True

    # 5. 主要計画／実績比較＋典型的カラム名
    if "主要計画" in s:
        if ("実績比較" in s
            or "項  目" in s
            or "計  画" in s
            or "実  績" in s):
            return True

    # 6. 表ヘッダーっぽい（句点なし＋複数行＋カラム語）
    if "。" not in s and len(lines) >= 3:
        header_keywords = ["項  目", "計  画", "実  績", "指標", "区分"]
        if any(k in s for k in header_keywords):
            return True

    return False

# 実際に適用
df_out = df_out[~df_out["text"].apply(is_noise_paragraph)].reset_index(drop=True)

In [ ]:
# レーティングを結合
df_out = df_out.merge(
    df[["file", "total_eval"]],
    left_on="source",
    right_on="file",
    how="left"
).drop(columns=["file"])

In [ ]:
df_out

,project_id,para_id,text,source,total_eval
0,2010_0200600_4_f.pdf,0,4.2 提言 \n4.2.1 実施機関への提言 \n事後評価時において、アティ橋、イクサ橋と...,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,4.0
1,2010_0200600_4_f.pdf,1,4.2.2 JICAへの提言 \nなし。,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,4.0
2,2010_0200600_4_f.pdf,2,4.3 教訓 \n事前評価時に設定された運用効果指標の大半は実施機関でデータ収集がなされてい...,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,4.0
3,2010_0202100_4_f.pdf,0,4．結論及び教訓・提言 \n4.1 結論 \n本事業は計画時、事業評価時点共に、ベトナム国の...,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,3.0
4,2010_0202100_4_f.pdf,1,4.2 提言 \n4.2.1 実施機関への提言 \n(1) 適正な維持管理を行うための提言（...,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,3.0
...,...,...,...,...,...
1088,2011_0602441_4_f.pdf,104,2) E/M 農家の普及活動の活用 \nE/M 農家を対象とした研修や現地視察などプロジェク...,https://www2.jica.go.jp/ja/evaluation/pdf/2011...,2.0
1089,2011_0602441_4_f.pdf,105,4.2.2 JICA への提言 \n特になし。,https://www2.jica.go.jp/ja/evaluation/pdf/2011...,2.0
1090,2011_0602441_4_f.pdf,106,4444....3333 教訓\n教訓\n教訓\n教訓 \n1) 新しいコンセプ...,https://www2.jica.go.jp/ja/evaluation/pdf/2011...,2.0
1091,2011_0602441_4_f.pdf,107,2) プロジェクト成果の普及のための既存制度の改善 \n本プロジェクトの成果であるADCやT...,https://www2.jica.go.jp/ja/evaluation/pdf/2011...,2.0


In [ ]:
df_out.to_csv('../kadai_text_with_rating.csv', index=False)

In [ ]:
import pandas as pd
df_out = pd.read_csv('../kadai_text_with_rating.csv')

## BERTで埋め込みベクトル作成

In [ ]:
%pip install "protobuf<3.21" --upgrade
%pip install -U "transformers" "sentence-transformers"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

texts = df_out["text"].tolist()
embeddings = model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)  # shape: (N, D)

print(embeddings.shape)

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

(1093, 384)


## クラスタリング

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# PCAで次元削減（例: 50次元）
pca_dim = 50
pca = PCA(n_components=pca_dim, random_state=42)
X_reduced = pca.fit_transform(embeddings)

# k-meansでクラスタリング
K = 20  # 後で変えて比較してもいい
km = KMeans(n_clusters=K, random_state=42, n_init="auto")
cluster_labels = km.fit_predict(X_reduced)

df_out["cluster"] = cluster_labels

In [ ]:
def is_bad(r):
    if pd.isna(r):
        return np.nan
    return 1 if r in [1.0, 2.0] else 0

df_out["bad_flag"] = df_out["total_eval"].apply(is_bad)

cluster_stats = (
    df_out.groupby("cluster")
      .agg(
          n=("text", "size"),
          bad_ratio=("bad_flag", "mean")
      )
      .reset_index()
      .sort_values("bad_ratio", ascending=False)
)

print(cluster_stats)

    cluster    n  bad_ratio
9         9   16   0.937500
11       11   87   0.793103
19       19    9   0.777778
14       14   81   0.456790
3         3   55   0.418182
12       12   30   0.333333
5         5   81   0.259259
10       10   91   0.252747
1         1  128   0.250000
17       17   61   0.245902
13       13   53   0.245283
7         7   29   0.241379
4         4   34   0.235294
16       16   58   0.224138
2         2   62   0.161290
18       18   15   0.133333
0         0   54   0.111111
15       15   10   0.100000
6         6   54   0.055556
8         8   85   0.047059


In [ ]:
top_clusters = cluster_stats.sort_values("bad_ratio", ascending=False).head(5)["cluster"].tolist()

for c in top_clusters:
    print(f"\n=== Cluster {c} ===")
    subset = df_out[df_out["cluster"] == c].sample(5, random_state=42)
    for t in subset["text"]:
        print("-", t.replace("\n", " "))


=== Cluster 9 ===
- 表6.	  Luang Prabang 県における森林率の推移  郡  森林率 (%)  1990 年付近  増減  2000 年付近  増減  2010 年付近  Luang Prabang 県のプロジェクト対象郡：  Viengkham 郡  68.2  -9.7  58.5  -4.6  53.9  Nan 郡  62.1  -9.1  -3.0  Pakseng 郡  61.3  -10.4  50.9  -9.1  41.8  プロジェクト対象以外の 郡の平均  62.4   -8.7  53.7   -5.9  47.8   出所：PAREDD 関連資料（衛星画像解析に基づく土地被覆動向の把握）を元に評価者が作 成
- り15にて、Nan 郡Pondong 村（IS）で103ha から75ha に縮小、Pakseng 郡Hat Houay 村（IS）で100ha から36ha に縮小、Sayaboury 郡Natak 村（1st PS）で78ha から36ha に縮小、Sayaboury 郡Tha 村（2nd PS）で280ha から180ha に縮小、Pakseng 郡Houasaking 村（3rd PS）で70ha から13ha に縮小していることが分かった（なお、焼畑耕作が縮 小している理由としては本プロジェクトの効果の他、地方によっては政策で焼畑が 禁止されている、一般に焼畑は労働量が多いことから代替的生計手段への転換が進 んでいる、Lao Policy Bank やLao Agriculture Promotion Bank の水田拡張資金を提供 するプロモーションの効果、なども影響しているという回答があった）。
- 陸稲栽培面積（’000 ha）  20.6  20.0  16.6  15.8  19.0  16.8  陸稲生産高（’000 Tons）  39.6  32.1  24.1  21.8  27.6  32.4  Sayaboury 県
- 表9：対象郡の主な農作物の収量（単位Kg/ha） *下線は全国平均を上回る数値  （Kg/ha）  出所：JICA 提供資料/ Statistical Information on Nepalese Agriculture 2010/2011
- ＜